In [ ]:
from __future__ import absolute_import, division, print_function

%matplotlib inline
# %matplotlib nbagg
import tensorflow as tf
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from IPython import display
from data_generator_tensorflow import get_batch, print_valid_characters

import os
import sys
sys.path.append(os.path.join('.', '..')) 
import utils 

import tf_utils

# Recurrent Neural Networks
> <span style="color:gray">
Original [Theano/Lasagne tutorial](https://github.com/DeepLearningDTU/nvidia_deep_learning_summercamp_2016/) by 
Lars Maaløe ([larsmaaloee](https://github.com/larsmaaloee)),
Søren Kaae Sønderby ([skaae](https://github.com/skaae)), and 
Casper Sønderby ([casperkaae](https://github.com/casperkaae)). 
Converted to TensorFlow by 
Alexander R. Johansen ([alrojo](https://github.com/alrojo)), 
and updated by 
Toke Faurby ([faur](https://github.com/Faur)).
> </span>

Recurrent neural networks (RNN) are the natural type of neural network to use for sequential data e.g. time series analysis, translation, speech recognition, biological sequence analysis etc.
RNNs works by recursively applying the same operation to an input $x_t$ and its own hidden state from the previous timestep $h_{t-1}$.
That is to say that each layer can be described by the function $f$:

$$y_t, h_t = f(x_t, h_{t-1})$$

where $y_t$ is the output.
An RNN can therefore handle input of varying length.

Drawing all the connections in a RNN (left) quickly becomes messy, therefore it is common to use the time unrolled view (right) when representing RNNs.
<img src='images/rnn_basic.png', width=600>
*Image by [Alex Graves](https://www.cs.toronto.edu/~graves/preprint.pdf)*



#### External resources
* The code describing RNNs can be tricky to understand at first. 
R2T2 has a great tutorial series ([part 1](https://r2rt.com/recurrent-neural-networks-in-tensorflow-i.html), [part 2](https://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html)) that digs into the details of how RNNs are implemented in TensorFlow. This introduction is heavily inspired by part 1.
* For more in depth background material on RNNs please see [Supervised Sequence Labelling with Recurrent
Neural Networks](https://www.cs.toronto.edu/~graves/preprint.pdf) by Alex Graves
* Lastly there is an [official TensorFlow tutorial](https://www.tensorflow.org/tutorials/recurrent) that is also worth a look

# Encoder-Decoder Models


Recurrent networks can be used for several kinds of prediction tasks including: 
* **One-to-one** - NOT a recurrent network. E.g. image classification.
* **One-to-many** - E.g. creating an [image caption](http://cs.stanford.edu/people/karpathy/deepimagesent/) with an RNN.
* **Many-to-one** - E.g. sentiment analysis of text.
* **Many-to-many** (different lengths) This is a combination of the *one-to-many* and *many-to-One*, and is called an **encoder-decoder** RNN. E.g. machine translation.
* **Many-to-many** (same length) Each input has an output. E.g. robotics control.


<img src="images/types.jpeg", width=800>

*Image courtesy Andrej Karpathy's [blog](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)*



## Encoder-Decoder
In this exercise we'll implement a Encoder-Decoder RNN based for a simple sequence to sequence translation task.
We will use a special kind of unit, called the GRU unit.
The GRU unit stores a hidden value per neuron that helps it '*remember*' long-term dependencies.
Another popular choice are LSTMs, which store two values, but these are approximately twice as slow as GRU.
GRUs (and LSTMs) are difficult to understand.
For a very good not-to-mathematical introduction see 
[Chris Olahs blog](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) or
[Andrej Karpathys blog](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)
(All their posts are nice and cover various topics within machine-learning).
This type of models have shown impressive performance in Neural Machine Translation and Image Caption generation. 


In the encoder-decoder structure one RNN (blue) encodes the input into a hidden representation, and a second RNN (red) uses this encoding to predict the target values.
An essential step is deciding how the encoder and decoder should communicate.
In the simplest approach you use the last hidden state of the encoder to initialize the decoder.
This is what we will do in this notebook, as shown here:

<img src='images/enc-dec.png', width=400>

#### Teacher forcing
We will also use a trick called _teacher forcing_ and that helps RNNs train faster.
The goal is to have the decoder predict the entire output sequence on its own.
However if a prediction in a sequence is wrong it is likely that the following predictions will also be wrong as the output at one timestep is used to predict the next (gray lines in the figure).
This slows learning, as the errors made after the first one aren't very informative.
In order to aliviate this we, at training time, feed the correct targets instead of the predictions.


#### Alternatives
There are other ways to let the encoder and decoder communicate with each other.
For instance you can give the last state of the Encoder as input to the Decoder at each decode time step, not just the previously predicted word.
Another approache is called **attention**, which lets the Decoder attend to different parts of the encoded input at different timesteps in the decoding process. 
Attention is shown in the next notebook. 



### The Data
Since RNN models can be very slow to train on real large datasets we will generate some simpler training data for this exercise. The task for the RNN is simply to translate a string of letters spelling the numbers between 0-9 into the corresponding numbers i.e

    "one two five" --> "125#"

`#` is a special end-of-sequence character, indicating the sequence is done.

To input the strings into the RNN model we translate the characters into a vector integers using a simple translation table (i.e. 'h'->16, 'o'-> 17 etc).
The code below prints a few input/output pairs using the *get_batch* function which randomy produces the data.


In the data loader below will setup the data and print some information. 
Key to understand are:
 * ENCODED INPUTS (`inputs`) are feed into the encoder (`A B C D` in the figure)
 * ENCODED TARGETS OUTPUT (`targets_out`) are what we want the network to predict. This is used to compute the error. (`X Y Z EOS` in the figure) 
 * ENCODED TARGETS INPUT (`targets_in`) are used for teacher forcing during training. (`EOS X Y Z` in the figure) 
 
Note; that we use the same symbol for end-of-sequence as for start-of-sequence (`#`).


In [ ]:
# At the bottom of the script there is some code which saves the model.
# If you wish to restore your model from a previous state use this function.
load_model = False

In [ ]:
batch_size = 3
inputs, inputs_seqlen, targets_in, targets_out, targets_seqlen, targets_mask, \
text_inputs, text_targets_in, text_targets_out = \
    get_batch(batch_size=batch_size, max_digits=4, min_digits=2)

print("input types:", inputs.dtype, inputs_seqlen.dtype, targets_in.dtype, targets_out.dtype, targets_seqlen.dtype)
print_valid_characters()
print("Stop/start character = #")

for i in range(batch_size):
    print("\nSAMPLE",i)
    print("TEXT INPUTS:\t\t\t", text_inputs[i])
    print("ENCODED INPUTS:\t\t\t", inputs[i])
    print("INPUTS SEQUENCE LENGTH:\t\t", inputs_seqlen[i])
    print("TEXT TARGETS OUTPUT:\t\t", text_targets_out[i])
    print("TEXT TARGETS INPUT:\t\t", text_targets_in[i])
    print("ENCODED TARGETS OUTPUT:\t\t", targets_out[i])
    print("ENCODED TARGETS INPUT:\t\t", targets_in[i])
    print("TARGETS SEQUENCE LENGTH:\t", targets_seqlen[i])
    print("TARGETS MASK:\t\t\t", targets_mask[i])

### Encoder-Decoder model setup
Below is the TensorFlow model definition. We use an embedding layer to go from integer representation to vector representation of the input.

TensorFlow has implementations of LSTM and GRU units.
Both implementations assume that the input from the tensor below has the shape **`[batch_size, max_time, input_size]`**, (unless you have `time_major=True`, in which case it is `[max_time, batch_size, input_size]`).

Note that we have made use of a custom decoder wrapper which can be found in `tf_utils.py`.

In [ ]:
# resetting the graph
tf.reset_default_graph()

# Setting up hyperparameters and general configs
MAX_DIGITS = 10
MIN_DIGITS = 5
NUM_INPUTS = 27
NUM_OUTPUTS = 11 #(0-9 + '#')

BATCH_SIZE = 16
# try various learning rates 1e-2 to 1e-5
LEARNING_RATE = 0.005
X_EMBEDDINGS = 8
t_EMBEDDINGS = 8
NUM_UNITS_ENC = 16
NUM_UNITS_DEC = 16


# Setting up placeholders, these are the tensors that we "feed" to our network
Xs = tf.placeholder(tf.int32, shape=[None, None], name='X_input')
ts_in = tf.placeholder(tf.int32, shape=[None, None], name='t_input_in')
ts_out = tf.placeholder(tf.int32, shape=[None, None], name='t_input_out')
X_len = tf.placeholder(tf.int32, shape=[None], name='X_len')
t_len = tf.placeholder(tf.int32, shape=[None], name='X_len')
t_mask = tf.placeholder(tf.float32, shape=[None, None], name='t_mask')


### Building the model
# first we build the embeddings to make our characters into dense, trainable vectors
X_embeddings = tf.get_variable('X_embeddings', [NUM_INPUTS, X_EMBEDDINGS],
                               initializer=tf.random_normal_initializer(stddev=0.1))
t_embeddings = tf.get_variable('t_embeddings', [NUM_OUTPUTS, t_EMBEDDINGS],
                               initializer=tf.random_normal_initializer(stddev=0.1))

X_embedded = tf.gather(X_embeddings, Xs, name='embed_X')
t_embedded = tf.gather(t_embeddings, ts_in, name='embed_t')


## forward encoding
enc_cell = tf.nn.rnn_cell.GRUCell(NUM_UNITS_ENC)
_, enc_state = tf.nn.dynamic_rnn(cell=enc_cell, inputs=X_embedded,
                                 sequence_length=X_len, dtype=tf.float32)
# use below incase TF's makes issues
# enc_state, _ = tf_utils.encoder(X_embedded, X_len, 'encoder', NUM_UNITS_ENC)
#
# enc_state = tf.concat(1, [enc_state, enc_state])

## decoding
# note that we are using a wrapper for decoding here, this wrapper is hardcoded to only use GRU
# check out tf_utils to see how you make your own decoder

# setting up weights for computing the final output
W_out = tf.get_variable('W_out', [NUM_UNITS_DEC, NUM_OUTPUTS])
b_out = tf.get_variable('b_out', [NUM_OUTPUTS])

dec_out, valid_dec_out = tf_utils.decoder(enc_state, t_embedded, t_len, 
                                          NUM_UNITS_DEC, t_embeddings,
                                          W_out, b_out)

## reshaping to have [batch_size*seqlen, num_units]
out_tensor = tf.reshape(dec_out, [-1, NUM_UNITS_DEC])
valid_out_tensor = tf.reshape(valid_dec_out, [-1, NUM_UNITS_DEC])
# computing output
out_tensor = tf.matmul(out_tensor, W_out) + b_out
valid_out_tensor = tf.matmul(valid_out_tensor, W_out) + b_out

## reshaping back to sequence
# print('X_len', tf.shape(X_len)[0])
b_size = tf.shape(X_len)[0] # use a variable we know has batch_size in [0]
seq_len = tf.shape(t_embedded)[1] # variable we know has sequence length in [1]
num_out = tf.constant(NUM_OUTPUTS) # casting NUM_OUTPUTS to a tensor variable
out_shape = tf.concat([tf.expand_dims(b_size, 0),
                      tf.expand_dims(seq_len, 0),
                      tf.expand_dims(num_out, 0)],
                     axis=0)

out_tensor = tf.reshape(out_tensor, out_shape)
valid_out_tensor = tf.reshape(valid_out_tensor, out_shape)
# handling shape loss
#out_tensor.set_shape([None, None, NUM_OUTPUTS])
y = out_tensor
y_valid = valid_out_tensor

In [ ]:
# print all the variable names and shapes
for var in tf.global_variables ():
    s = var.name + " "*(40-len(var.name))
    print(s, var.value().get_shape())

### Defining the cost function, gradient clipping and accuracy
Because the targets are categorical we use the cross entropy error.
As the data is sequential we use the sequence to sequence cross entropy supplied in `tf_utils.py`.
We use the Adam optimizer but you can experiment with the different optimizers implemented in [TensorFlow](https://www.tensorflow.org/api_docs/python/tf/train/Optimizer).

In [ ]:
def loss_and_acc(preds):
    # sequence_loss_tensor is a modification of TensorFlow's own sequence_to_sequence_loss
    # TensorFlow's seq2seq loss works with a 2D list instead of a 3D tensors
    loss = tf_utils.sequence_loss_tensor(preds, ts_out, t_mask, NUM_OUTPUTS) # notice that we use ts_out here!

    ## if you want regularization
    #reg_scale = 0.00001
    #regularize = tf.contrib.layers.l2_regularizer(reg_scale)
    #params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
    #reg_term = sum([regularize(param) for param in params])
    #loss += reg_term
    
    ## calculate accuracy
    argmax = tf.to_int32(tf.argmax(preds, 2))
    correct = tf.to_float(tf.equal(argmax, ts_out)) * t_mask
    accuracy = tf.reduce_sum(correct) / tf.reduce_sum(t_mask)
    return loss, accuracy, argmax

loss, accuracy, predictions = loss_and_acc(y)
loss_valid, accuracy_valid, predictions_valid = loss_and_acc(y_valid)

# use lobal step to keep track of our iterations
global_step = tf.Variable(0, name='global_step', trainable=False)

# pick optimizer, try momentum or adadelta
optimizer = tf.train.AdamOptimizer(LEARNING_RATE)

# extract gradients for each variable
grads_and_vars = optimizer.compute_gradients(loss)

## add below for clipping by norm
# gradients, variables = zip(*grads_and_vars)  # unzip list of tuples
# clipped_gradients, global_norm = (
#    tf.clip_by_global_norm(gradients, self.clip_norm) )
# grads_and_vars = zip(clipped_gradients, variables)
# apply gradients and make trainable function
train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

In [ ]:
# print all the variable names and shapes
# notice that we now have the optimizer Adam as well!
for var in tf.all_variables():
    s = var.name + " "*(40-len(var.name))
    print(s, var.value().get_shape())

### Testing the forward pass

In [ ]:
## Start the session
# restricting memory usage, TensorFlow is greedy and will use all memory otherwise
gpu_opts = tf.GPUOptions(per_process_gpu_memory_fraction=0.35)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_opts))

# Initialize parameters
if load_model:
    try:
        tf.train.Saver().restore(sess, "/save/model.ckpt")
    except:
        sess.run(tf.global_variables_initializer())
        print('Model not found, new parameters initialized')
else:
    sess.run(tf.global_variables_initializer())

In [ ]:
# as always, test the forward pass and initialize the tf.Session!
for i in range(batch_size):
    print("\nSAMPLE",i)
    print("TEXT INPUTS:\t\t\t", text_inputs[i])
    print("TEXT TARGETS INPUT:\t\t", text_targets_in[i])

feed_dict = {Xs: inputs, X_len: inputs_seqlen, ts_in: targets_in,
             ts_out: targets_out, t_len: targets_seqlen}

# test training forwardpass
fetches = [y]
res = sess.run(fetches=fetches, feed_dict=feed_dict)
print("y", res[0].shape)

# test validation forwardpass
fetches = [y_valid]
res = sess.run(fetches=fetches, feed_dict=feed_dict)
print("y_valid", res[0].shape)

In [ ]:
#Generate some validation data
X_val, X_len_val, t_in_val, t_out_val, t_len_val, t_mask_val, \
text_inputs_val, text_targets_in_val, text_targets_out_val = \
    get_batch(batch_size=5000, max_digits=MAX_DIGITS, min_digits=MIN_DIGITS)
print("X_val", X_val.shape)
print("t_out_val", t_out_val.shape)

# Training

Training RNN can take a while, especially if you are running it on your laptop.
We won't train the model to completion, as the trends we are interested in can be seen earlier.
If training takes to long feel free to stop it even earlier by interrupting the kernel.

In [ ]:
%%time
## If you get an error, remove this line! It makes the error message hard to understand.

# setting up running parameters
val_interval = 5000
samples_to_process = 2e5
samples_processed = 0
samples_val = []
costs, accs_val = [], []
plt.figure()
try:
    while samples_processed < samples_to_process:
        # load data
        X_tr, X_len_tr, t_in_tr, t_out_tr, t_len_tr, t_mask_tr, \
        text_inputs_tr, text_targets_in_tr, text_targets_out_tr = \
            get_batch(batch_size=BATCH_SIZE,max_digits=MAX_DIGITS,min_digits=MIN_DIGITS)
        # make fetches
        fetches_tr = [train_op, loss, accuracy]
        # set up feed dict
        feed_dict_tr = {Xs: X_tr, X_len: X_len_tr, ts_in: t_in_tr,
             ts_out: t_out_tr, t_len: t_len_tr, t_mask: t_mask_tr}
        # run the model
        res = tuple(sess.run(fetches=fetches_tr, feed_dict=feed_dict_tr))
        _, batch_cost, batch_acc = res
        costs += [batch_cost]
        samples_processed += BATCH_SIZE
        #if samples_processed % 1000 == 0: print(batch_cost, batch_acc)
        #validation data
        if samples_processed % val_interval == 0:
            #print("validating")
            fetches_val = [accuracy_valid, y_valid]
            feed_dict_val = {Xs: X_val, X_len: X_len_val, ts_in: t_in_val,
             ts_out: t_out_val, t_len: t_len_val, t_mask: t_mask_val}
            res = tuple(sess.run(fetches=fetches_val, feed_dict=feed_dict_val))
            acc_val, output_val = res
            samples_val += [samples_processed]
            accs_val += [acc_val]
            plt.plot(samples_val, accs_val, 'g-')
            plt.ylabel('Validation Accuracy', fontsize=15)
            plt.xlabel('Processed samples', fontsize=15)
            plt.title('', fontsize=20)
            plt.grid('on')
            plt.savefig("out.png")
            display.display(display.Image(filename="out.png"))
            display.clear_output(wait=True)
except KeyboardInterrupt:
    pass

print('Done')

In [ ]:
#plot of validation accuracy for each target position
plt.figure(figsize=(7,7))
plt.plot(np.mean(np.argmax(output_val,axis=2)==t_out_val,axis=0))
plt.ylabel('Accuracy', fontsize=15)
plt.xlabel('Target position', fontsize=15)
#plt.title('', fontsize=20)
plt.grid('on')
plt.show()
#why do the plot look like this?

In [ ]:
## Save model
# Read more about saving and loading models at https://www.tensorflow.org/programmers_guide/saved_model

# Save model
save_path = tf.train.Saver().save(sess, "/tmp/model.ckpt")
print("Model saved in file: %s" % save_path)


In [ ]:
## Close the session, and free the resources
sess.close()

# Exercises:


### Exercise 1)
1. What is the final validation performance? 
1. Why do you think it is not better?
1. Comment on the accuracy for each position in of the output symbols?

___
Answer:


### Exercise 2)
The model has two GRU networks. The ```GRUEncoder``` and the ```GRUDecoder```.
A GRU is parameterized by a update gate $u$, a  reset gate $r$, the cell $c$, and the hidden state $h$:

![](images/GRUeq.png)
*Equations as described in the [Lasagne GRU documentation](http://lasagne.readthedocs.io/en/latest/modules/layers/recurrent.html#lasagne.layers.GRULayer).*

**Note** that the notation in the implementation (`tf_utils.py`) $z$ is used instead of $u$.

Under normal circumstances, such as in the TensorFlow GRUCell implementation, these gates have been stacked for faster computation, but in the custom decoder each weight and bias are as described in the original [article for GRU](https://arxiv.org/abs/1406.1078).
 1. Try to explain the shape of $W_{xr}$ and $W_{hr}$.
 1. Why are they different? 

___
Answer:

### Exercise 3)
The GRU-unit is able to ignore the input and just copy the previous hidden state.
In the beginning of training this might be desireable behaviour because it helps the model learn long range dependencies.
You can make the model ignore the input by modifying initial bias values.
1. What bias would you modify and how would you modify it?

Again you'll need to refer to the [GRU equations](http://lasagne.readthedocs.io/en/latest/modules/layers/recurrent.html#lasagne.layers.GRULayer)
Further, if you look into `tf_utils.py` and search for the `decoder(...)` function, you will see that the init for each weight and bias can be changed.

___
Answer:



### Exercise 4)
In the example we stack a softmax layer on top of a Recurrent layer. In the code snippet below explain how we can do that?
___
Answer:

In [ ]:
tf.reset_default_graph()

bs_, seqlen_, numinputs_ = 16, 140, 40 # Batch_size, Sequence_length, number_of_inputs
x_pl_ = tf.placeholder(tf.float32, [bs_, seqlen_, numinputs_])
gru_cell_ = tf.nn.rnn_cell.GRUCell(10)
l_gru_, gru_state_ = tf.nn.dynamic_rnn(gru_cell_, x_pl_, dtype=tf.float32)
l_reshape_ = tf.reshape(l_gru_, [-1, 10])

l_softmax_ = tf.contrib.layers.fully_connected(l_reshape_, 11, activation_fn=tf.nn.softmax)
l_softmax_seq_ = tf.reshape(l_softmax_, [bs_, seqlen_, -1])

print("l_input_\t", x_pl_.get_shape())
print("l_gru_\t\t", l_gru_.get_shape())
print("l_reshape_\t", l_reshape_.get_shape())
print("l_softmax_\t", l_softmax_.get_shape())
print("l_softmax_seq_\t", l_softmax_seq_.get_shape())

### Optional Exercise 1)
Why do you think the validation performance looks more "jig-saw" like compared to FFN and CNN models?

___
Answer:

### Optional Exercise 2)
You are interested in doing sentiment analysis on tweets, i.e classification as positive or negative. You decide read over the twitter seqeuence and use the last hidden state to do the classification. How can you modify the small network above to only output a single classification for network? Hints: look at the gru\_state\_ or the [tf.slice](https://www.tensorflow.org/versions/r0.10/api_docs/python/array_ops.html#slice) in the API.




### Optional Exercise 3)
Bidirectional Encoders are usually implemented by running a forward model and  a backward model (a forward model on a reversed sequence) separately and the concatenating them before parsing them on to the next layer. To reverse the sequence try looking at [tf.reverse_sequence](https://www.tensorflow.org/versions/r0.10/api_docs/python/array_ops.html#reverse_sequence)

Implement a Bidirectional encoder model.
Here is some code to get you started:

``` python
enc_cell = tf.nn.rnn_cell.GRUCell(NUM_UNITS_ENC)
_, enc_state = tf.nn.dynamic_rnn(cell=enc_cell, inputs=X_embedded,
                                 sequence_length=X_len, dtype=tf.float32, scope="rnn_forward")
X_embedded_backwards = tf.reverse_sequence(X_embedded, tf.to_int64(X_len), 1)
enc_cell_backwards = tf.nn.rnn_cell.GRUCell(NUM_UNITS_ENC)
_, enc_state_backwards = tf.nn.dynamic_rnn(cell=enc_cell_backwards, inputs=X_embedded_backwards,
                                 sequence_length=X_len, dtype=tf.float32, scope="rnn_backward")

enc_state = tf.concat(1, [enc_state, enc_state_backwards])
```

Note: you will need to double the NUM_UNITS_DEC, as it currently does not support different sizes.